In [1]:
import os
from tqdm import tqdm
from itertools import chain
import pandas as pd
import numpy as np
import random, torch
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style = 'white', font_scale=1.5)

import sys
sys.path.insert(0, '/home/was966/Research/mims-conceptor/')
from conceptor.utils import plot_embed_with_label
from conceptor import PreTrainer, FineTuner, loadconceptor
from conceptor.utils import plot_embed_with_label, score
from conceptor.tokenizer import CANCER_CODE


def onehot(S):
    assert type(S) == pd.Series, 'Input type should be pd.Series'
    dfd = pd.get_dummies(S, dummy_na=True)
    nanidx = dfd[dfd[np.nan]].index
    dfd.loc[nanidx, :] = np.nan
    dfd = dfd.drop(columns=[np.nan])*1.
    cols = dfd.sum().sort_values(ascending=False).index.tolist()
    dfd = dfd[cols]
    return dfd

In [2]:
data_path = '/n/data1/hms/dbmi/zitnik/lab/users/was966/ITRP/TCGA-ITRP-GENESET-MERGE/15672//data'

df_tpm = pd.read_pickle(os.path.join(data_path,  'TCGA.TPM.TABLE'))
df_label = pd.read_pickle(os.path.join(data_path, 'TCGA.PATIENT.PROCESSED.TABLE'))
df_label_org = pd.read_pickle(os.path.join(data_path, 'TCGA.PATIENT.TABLE'))
df_tpm.shape, df_label.shape

((10184, 15672), (10184, 13))

In [3]:
# load the pretrained model as a feature extractor
pretrainer = loadconceptor('../checkpoint/latest/pretrainer.pt')
genesetprojector = pretrainer.model.latentprojector.genesetprojector
cellpathwayprojector = pretrainer.model.latentprojector.cellpathwayprojector
df_tpm = df_tpm[pretrainer.feature_name]
pretrainer.count_parameters()

1019894

In [4]:
dfcx = df_label.cancer_type.map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)
dfgeneset, dfcelltype = pretrainer.extract(dfcx,  batch_size= 128)

100%|##########| 80/80 [00:14<00:00,  5.63it/s]


In [5]:
df_label_org.to_csv('./TCGA/00_clinical_label_orignal.csv')
df_label.to_csv('./TCGA/00_clinical_label.csv')
dfgeneset.to_csv('./TCGA/01_readouts_geneset.csv')
dfcelltype.to_csv('./TCGA/02_readouts_celltype.csv')

In [6]:
dfg, dfc = pretrainer.project(dfcx,  batch_size= 128)

100%|##########| 80/80 [00:24<00:00,  3.20it/s]


In [7]:
pid = dfg.index.map(lambda x:x.split('$$')[0])
fid = dfg.index.map(lambda x:x.split('$$')[1])
df = pd.DataFrame(index=dfg.index)
df['bcr_patient_barcode'] = pid
df['feature_name'] = fid
df = df.join(dfg)
df = df.sort_values(['feature_name', 'bcr_patient_barcode'])
df.to_csv('./TCGA/03_features_geneset.csv')

In [8]:
pid = dfc.index.map(lambda x:x.split('$$')[0])
fid = dfc.index.map(lambda x:x.split('$$')[1])
df = pd.DataFrame(index=dfc.index)
df['bcr_patient_barcode'] = pid
df['feature_name'] = fid
df = df.join(dfc)
df = df.sort_values(['feature_name', 'bcr_patient_barcode'])
df.to_csv('./TCGA/04_features_celltype.csv')